In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Define the Dataset class
class CSVDataset(Dataset):
    def __init__(self, filepath):
        self.data = pd.read_csv(filepath)
        self.X = torch.from_numpy(self.data.iloc[:, :-1].values).float()
        self.y = torch.from_numpy(self.data.iloc[:, -1].values).unsqueeze(1).float()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the neural network class
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Assume input features are 10 as per the synthetic data
        self.fc1 = torch.nn.Linear(10, 8)
        self.fc2 = torch.nn.Linear(8, 6)
        self.fc3 = torch.nn.Linear(6, 4)
        self.fc4 = torch.nn.Linear(4, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Load the dataset
# dataset = CSVDataset(filepath='synthetic_data.csv')
# 
# # Create a DataLoader
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# 
# # Instantiate the model
# model = Net()
# 
# # Define loss function and optimizer
# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 
# # Training loop
# for epoch in range(5):  # let's say we run for 5 epochs
#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
#         inputs, labels = data
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     print(f'Epoch {epoch+1}: loss={running_loss/len(dataloader)}')

# Save the trained model
#torch.save(model.state_dict(), '/model.pth')

In [13]:
def train(dataloader, model, optimizer, criterion, epochs):
    for epoch in range(5):  # let's say we run for 5 epochs
        running_loss = 0.0
        for i, data in enumerate(dataloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}: loss={running_loss/len(dataloader)}')

In [14]:
def test_train():
    # Load the dataset
    dataset = CSVDataset(filepath='synthetic_data.csv')
    
    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Instantiate the model
    model = Net()
    
    # Define loss function and optimizer
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train(dataloader, model, optimizer, criterion, epochs=5)

In [15]:
test_train()

Epoch 1: loss=0.6968666482716799
Epoch 2: loss=0.6941703725606203
Epoch 3: loss=0.6932755429297686
Epoch 4: loss=0.6924158092588186
Epoch 5: loss=0.6919758729636669
